# Exercise 9: Understanding MCP Tools
## Creating and Consuming MCP Tools with FastMCP and LangChain

In this notebook, we'll learn how to create MCP (Model Context Protocol) tools using FastMCP and how to consume them in LangChain for financial data processing.

## Learning Objectives
- Understand MCP concepts and architecture
- Create MCP tools using FastMCP framework
- Set up mock MCP server for financial data
- Load and test MCP tools in LangChain
- Handle async operations with MCP tools

Let's explore the power of MCP tools for financial applications! 🔧📊💰

## Setup and Imports

First, let's install the required packages and set up our environment.

In [1]:
# Install required packages
!uv add fastmcp langchain langchain-openai mcp python-dotenv scikit-learn joblib requests numpy pandas

Resolved 392 packages in 7.08s                                       
⠙ pyobjc-framework-metrickit==11.1                                              Audited 223 packages in 8ms


In [2]:
# Import necessary libraries
import os
import json
import asyncio
import logging
from typing import List, Tuple, Dict, Any
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import Tool

# Load environment variables
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ Setup complete!")
print("✓ Ready to explore MCP tools!")

✓ Setup complete!
✓ Ready to explore MCP tools!


## Part 1: Understanding MCP Architecture

MCP (Model Context Protocol) enables AI assistants to securely connect to external tools and data sources. Let's understand the key concepts.

In [3]:
# MCP Architecture Overview
print("🏗️ MCP ARCHITECTURE CONCEPTS")
print("=" * 50)

mcp_concepts = {
    "MCP Server": {
        "description": "Exposes tools and resources to clients",
        "role": "Provides secure access to external capabilities",
        "example": "Financial data server with ML models"
    },
    "MCP Client": {
        "description": "Consumes tools from MCP servers",
        "role": "LangChain agent that uses MCP tools",
        "example": "AI assistant using financial tools"
    },
    "Tools": {
        "description": "Functions exposed by MCP server",
        "role": "Specific capabilities (predict, analyze, etc.)",
        "example": "train_model(), predict_shipment()"
    },
    "Resources": {
        "description": "Data sources exposed by server",
        "role": "Access to files, databases, APIs",
        "example": "Financial datasets, market data"
    },
    "Transport": {
        "description": "Communication protocol",
        "role": "HTTP, WebSocket, or other protocols",
        "example": "streamable-http on port 8000"
    }
}

for concept, details in mcp_concepts.items():
    print(f"\n🔹 {concept}:")
    print(f"   What: {details['description']}")
    print(f"   Role: {details['role']}")
    print(f"   Example: {details['example']}")

print("\n✅ MCP architecture concepts covered!")

🏗️ MCP ARCHITECTURE CONCEPTS

🔹 MCP Server:
   What: Exposes tools and resources to clients
   Role: Provides secure access to external capabilities
   Example: Financial data server with ML models

🔹 MCP Client:
   What: Consumes tools from MCP servers
   Role: LangChain agent that uses MCP tools
   Example: AI assistant using financial tools

🔹 Tools:
   What: Functions exposed by MCP server
   Role: Specific capabilities (predict, analyze, etc.)
   Example: train_model(), predict_shipment()

🔹 Resources:
   What: Data sources exposed by server
   Role: Access to files, databases, APIs
   Example: Financial datasets, market data

🔹 Transport:
   What: Communication protocol
   Role: HTTP, WebSocket, or other protocols
   Example: streamable-http on port 8000

✅ MCP architecture concepts covered!


## Part 2: Creating MCP Tools with FastMCP

Let's examine how to create MCP tools using the FastMCP framework, based on our existing `mcp_tool.py`.

In [4]:
# FastMCP Tool Creation Pattern
print("🛠️ FASTMCP TOOL CREATION PATTERNS")
print("=" * 50)

# Show the pattern for creating MCP tools
mcp_tool_pattern = '''
from fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP(name="financial-tools")

@mcp.tool()
def financial_analysis(data: List[Tuple[int, float]]) -> str:
    """Analyze financial data and return insights.
    
    Args:
        data: List of (timestamp, value) tuples
        
    Returns:
        str: Analysis results
    """
    # Tool implementation
    return "Analysis complete"

# Run the server
if __name__ == "__main__":
    mcp.run(transport="streamable-http", port=8000)
'''

print("📝 FastMCP Tool Pattern:")
print(mcp_tool_pattern)

print("\n🔑 Key FastMCP Features:")
features = [
    "Simple @mcp.tool() decorator",
    "Automatic type validation",
    "Built-in HTTP transport",
    "Async support",
    "Automatic OpenAPI documentation",
    "Error handling"
]

for i, feature in enumerate(features, 1):
    print(f"  {i}. {feature}")

print("\n✅ FastMCP patterns explained!")

🛠️ FASTMCP TOOL CREATION PATTERNS
📝 FastMCP Tool Pattern:

from fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP(name="financial-tools")

@mcp.tool()
def financial_analysis(data: List[Tuple[int, float]]) -> str:
    """Analyze financial data and return insights.

    Args:
        data: List of (timestamp, value) tuples

    Returns:
        str: Analysis results
    """
    # Tool implementation
    return "Analysis complete"

# Run the server
if __name__ == "__main__":
    mcp.run(transport="streamable-http", port=8000)


🔑 Key FastMCP Features:
  1. Simple @mcp.tool() decorator
  2. Automatic type validation
  3. Built-in HTTP transport
  4. Async support
  5. Automatic OpenAPI documentation
  6. Error handling

✅ FastMCP patterns explained!


## Part 3: Financial MCP Tools Implementation

Let's create a simplified version of our financial MCP tools for demonstration.

In [5]:
# Create a mock financial MCP server (simplified version)
from fastmcp import FastMCP
import joblib
from sklearn.linear_model import LinearRegression

# Mock MCP server for demonstration
class MockFinancialMCP:
    """Mock MCP server for financial tools demonstration."""
    
    def __init__(self):
        self.name = "financial-tools"
        self.model = None
        self.trained = False
        
    def train_financial_model(self, data: List[Tuple[int, float]]) -> str:
        """Train a financial prediction model.
        
        Args:
            data: List of (timestamp, value) tuples
            
        Returns:
            str: Training status
        """
        if len(data) < 5:
            return "Error: Need at least 5 data points"
        
        # Prepare data
        X = np.array([t for t, _ in data]).reshape(-1, 1)
        y = np.array([v for _, v in data])
        
        # Train model
        self.model = LinearRegression().fit(X, y)
        self.trained = True
        
        logger.info(f"Model trained on {len(data)} points")
        return f"✅ Financial model trained on {len(data)} data points"
    
    def predict_financial_trend(self, timestamp: int) -> str:
        """Predict financial value for given timestamp.
        
        Args:
            timestamp: Future timestamp to predict
            
        Returns:
            str: Prediction result
        """
        if not self.trained:
            return "Error: Model not trained yet"
        
        prediction = self.model.predict([[timestamp]])[0]
        logger.info(f"Predicted for t={timestamp}: {prediction:.2f}")
        return f"📈 Predicted value for t={timestamp}: {prediction:.2f}"
    
    def analyze_financial_data(self, data: List[float]) -> str:
        """Analyze financial data and provide insights.
        
        Args:
            data: List of financial values
            
        Returns:
            str: Analysis results
        """
        if not data:
            return "Error: No data provided"
        
        mean_val = np.mean(data)
        std_val = np.std(data)
        trend = "increasing" if data[-1] > data[0] else "decreasing"
        
        analysis = f"""
📊 Financial Data Analysis:
- Mean: {mean_val:.2f}
- Std Dev: {std_val:.2f}
- Trend: {trend}
- Data points: {len(data)}
- Range: {min(data):.2f} to {max(data):.2f}
"""
        return analysis

# Create mock MCP server instance
mock_mcp = MockFinancialMCP()

print("✅ Mock Financial MCP Server Created!")
print("  🏦 train_financial_model() - Train prediction models")
print("  📈 predict_financial_trend() - Make predictions")
print("  📊 analyze_financial_data() - Analyze data")

✅ Mock Financial MCP Server Created!
  🏦 train_financial_model() - Train prediction models
  📈 predict_financial_trend() - Make predictions
  📊 analyze_financial_data() - Analyze data


## Part 4: Converting MCP Tools to LangChain Tools

Now let's convert our MCP tools into LangChain tools that can be used by agents.

In [6]:
# Convert MCP tools to LangChain tools
def create_langchain_tools_from_mcp(mcp_server):
    """Convert MCP tools to LangChain tools."""
    
    # Tool 1: Train Financial Model
    def train_model_tool(data_json: str) -> str:
        """Train financial prediction model with data.
        
        Args:
            data_json: JSON string of [[timestamp, value], ...] pairs
        """
        try:
            data = json.loads(data_json)
            data_tuples = [(int(t), float(v)) for t, v in data]
            return mcp_server.train_financial_model(data_tuples)
        except Exception as e:
            return f"Error: {str(e)}"
    
    # Tool 2: Predict Financial Trend
    def predict_trend_tool(timestamp: str) -> str:
        """Predict financial value for future timestamp.
        
        Args:
            timestamp: Future timestamp as string
        """
        try:
            ts = int(timestamp)
            return mcp_server.predict_financial_trend(ts)
        except Exception as e:
            return f"Error: {str(e)}"
    
    # Tool 3: Analyze Financial Data
    def analyze_data_tool(data_json: str) -> str:
        """Analyze financial data and provide insights.
        
        Args:
            data_json: JSON string of [value1, value2, ...] array
        """
        try:
            data = json.loads(data_json)
            return mcp_server.analyze_financial_data(data)
        except Exception as e:
            return f"Error: {str(e)}"
    
    # Create LangChain tools
    tools = [
        Tool(
            name="train_financial_model",
            func=train_model_tool,
            description="Train a financial prediction model with timestamp-value pairs. Input: JSON array of [timestamp, value] pairs."
        ),
        Tool(
            name="predict_financial_trend", 
            func=predict_trend_tool,
            description="Predict financial value for a future timestamp. Input: timestamp as string."
        ),
        Tool(
            name="analyze_financial_data",
            func=analyze_data_tool,
            description="Analyze financial data and provide statistical insights. Input: JSON array of values."
        )
    ]
    
    return tools

# Create LangChain tools from our mock MCP server
financial_tools = create_langchain_tools_from_mcp(mock_mcp)

print("✅ LangChain Tools Created from MCP Server!")
print(f"  📊 Available tools: {len(financial_tools)}")
for tool in financial_tools:
    print(f"  🔧 {tool.name}: {tool.description[:50]}...")

✅ LangChain Tools Created from MCP Server!
  📊 Available tools: 3
  🔧 train_financial_model: Train a financial prediction model with timestamp-...
  🔧 predict_financial_trend: Predict financial value for a future timestamp. In...
  🔧 analyze_financial_data: Analyze financial data and provide statistical ins...


## Part 5: Creating Financial Analysis Agent

Let's create a LangChain agent that uses our MCP tools for financial analysis.

In [7]:
# Create Financial Analysis Agent with MCP tools
def create_financial_agent():
    """Create a financial analysis agent with MCP tools."""
    
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.1,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a Financial Analysis Agent with access to specialized MCP tools.
        
        Your tools:
        1. train_financial_model: Train prediction models on historical data
        2. predict_financial_trend: Make predictions for future timestamps
        3. analyze_financial_data: Analyze data and provide insights
        
        When using tools:
        - Always format data as proper JSON
        - Train models before making predictions
        - Provide clear, actionable insights
        - Handle errors gracefully
        
        Be concise and professional in your responses."""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    agent = create_tool_calling_agent(llm, financial_tools, prompt)
    return AgentExecutor(agent=agent, tools=financial_tools, verbose=True, max_iterations=3)

# Create the agent
financial_agent = create_financial_agent()

print("✅ Financial Analysis Agent Created!")
print("  🤖 Agent equipped with MCP tools")
print("  📊 Ready for financial analysis tasks")
print("  🔧 Tools: train, predict, analyze")

✅ Financial Analysis Agent Created!
  🤖 Agent equipped with MCP tools
  📊 Ready for financial analysis tasks
  🔧 Tools: train, predict, analyze


## Part 6: Testing MCP Tools Integration

Let's test our MCP tools integration with realistic financial scenarios.

In [8]:
# Test 1: Basic Financial Analysis
print("🧪 TEST 1: BASIC FINANCIAL ANALYSIS")
print("=" * 50)

# Test data analysis
test_query_1 = """
Analyze this financial data: [100, 105, 103, 108, 112, 115, 118, 120, 125, 130]
Provide insights about the trend and statistics.
"""

result_1 = financial_agent.invoke({"input": test_query_1})
print("📊 Analysis Result:")
print(result_1["output"])

print("\n" + "=" * 50)
print("🧪 TEST 2: MODEL TRAINING AND PREDICTION")
print("=" * 50)

# Test model training and prediction
test_query_2 = """
Train a financial model with this historical data: [[1, 100], [2, 105], [3, 110], [4, 115], [5, 120]]
Then predict the value for timestamp 6.
"""

result_2 = financial_agent.invoke({"input": test_query_2})
print("📈 Training and Prediction Result:")
print(result_2["output"])

print("\n✅ MCP Tools Integration Tests Completed!")

🧪 TEST 1: BASIC FINANCIAL ANALYSIS


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `analyze_financial_data` with `[100, 105, 103, 108, 112, 115, 118, 120, 125, 130]`



📊 Financial Data Analysis:
- Mean: 113.60
- Std Dev: 9.31
- Trend: increasing
- Data points: 10
- Range: 100.00 to 130.00


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The financial data analysis reveals the following insights:
- Mean value: 113.60
- Standard deviation: 9.31
- Trend: Increasing
- Number of data points: 10
- Range: 100.00 to 130.00

Overall, the data shows an increasing trend with a mean value of 113.60 and a standard deviation of 9.31.

> Finished chain.
📊 Analysis Result:
The financial data analysis reveals the following insights:
- Mean value: 113.60
- Standard deviation: 9.31
- Trend: Increasing
- Number of data points: 10
- Range: 100.00 to 130.00

Overall, the data shows an increasing trend with a mean value of 113.60 and a standard deviation of 9.31.

🧪 TEST 2: MODEL TRAINING AND PREDICTION


> Entering new AgentExecutor chain...


...


Invoking: `train_financial_model` with `[[1, 100], [2, 105], [3, 110], [4, 115], [5, 120]]`


✅ Financial model trained on 5 data points

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Predicted for t=6: 125.00



Invoking: `predict_financial_trend` with `6`


📈 Predicted value for t=6: 125.00

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The financial model has been successfully trained on the historical data provided. The predicted value for timestamp 6 is $125.00.

> Finished chain.
📈 Training and Prediction Result:
The financial model has been successfully trained on the historical data provided. The predicted value for timestamp 6 is $125.00.

✅ MCP Tools Integration Tests Completed!


## Part 7: Advanced MCP Usage Patterns

Let's explore advanced patterns for using MCP tools in financial applications.

In [9]:
# Advanced MCP Usage Patterns
print("🚀 ADVANCED MCP USAGE PATTERNS")
print("=" * 50)

# Pattern 1: Chained MCP Operations
def demonstrate_chained_operations():
    """Demonstrate chaining multiple MCP operations."""
    
    print("🔗 Pattern 1: Chained MCP Operations")
    
    # Generate sample financial data
    timestamps = list(range(1, 11))
    values = [100 + i * 2 + np.random.normal(0, 1) for i in range(10)]
    
    # Chain operations: Analyze -> Train -> Predict
    chain_query = f"""
    Please perform these operations in sequence:
    1. Analyze the data: {values}
    2. Train a model with timestamps and values: {list(zip(timestamps, values))}
    3. Predict values for timestamps 11 and 12
    
    Provide a comprehensive report.
    """
    
    result = financial_agent.invoke({"input": chain_query})
    print("📊 Chained Operations Result:")
    print(result["output"][:200] + "...")
    
    return result

# Pattern 2: Error Handling
def demonstrate_error_handling():
    """Demonstrate error handling in MCP tools."""
    
    print("\n🛡️ Pattern 2: Error Handling")
    
    # Test with insufficient data
    error_query = "Train a model with only 2 data points: [[1, 100], [2, 105]]"
    
    result = financial_agent.invoke({"input": error_query})
    print("⚠️ Error Handling Result:")
    print(result["output"])
    
    return result

# Pattern 3: Complex Analysis
def demonstrate_complex_analysis():
    """Demonstrate complex financial analysis."""
    
    print("\n📈 Pattern 3: Complex Analysis")
    
    # Multi-step analysis
    complex_query = """
    I have quarterly revenue data: [1000000, 1100000, 1050000, 1200000, 1250000, 1300000]
    
    Please:
    1. Analyze this data for trends and patterns
    2. Train a prediction model 
    3. Predict the next 2 quarters
    4. Provide business insights and recommendations
    """
    
    result = financial_agent.invoke({"input": complex_query})
    print("🎯 Complex Analysis Result:")
    print(result["output"][:200] + "...")
    
    return result

# Run demonstrations
demo_1 = demonstrate_chained_operations()
demo_2 = demonstrate_error_handling()
demo_3 = demonstrate_complex_analysis()

print("\n✅ Advanced MCP patterns demonstrated!")

🚀 ADVANCED MCP USAGE PATTERNS
🔗 Pattern 1: Chained MCP Operations


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `analyze_financial_data` with `[100.08184950962846, 101.7672787717486, 103.8968983189885, 104.8246269227791, 106.73396271058272, 110.41458499118345, 112.37522934666471, 113.65754112615409, 114.63106505487508, 119.3092494222631]`



📊 Financial Data Analysis:
- Mean: 108.77
- Std Dev: 5.94
- Trend: increasing
- Data points: 10
- Range: 100.08 to 119.31


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Predicted for t=11: 150.00
INFO:__main__:Predicted for t=12: 155.00



Invoking: `train_financial_model` with `[(1, 100.08184950962846), (2, 101.7672787717486), (3, 103.8968983189885), (4, 104.8246269227791), (5, 106.73396271058272), (6, 110.41458499118345), (7, 112.37522934666471), (8, 113.65754112615409), (9, 114.63106505487508), (10, 119.3092494222631)`


Error: Expecting value: line 1 column 2 (char 1)
Invoking: `predict_financial_trend` with `11`


📈 Predicted value for t=11: 150.00
Invoking: `predict_financial_trend` with `12`


📈 Predicted value for t=12: 155.00

...


Invoking: `analyze_financial_data` with `[1000000, 1100000, 1050000, 1200000, 1250000, 1300000]`



📊 Financial Data Analysis:
- Mean: 1150000.00
- Std Dev: 108012.34
- Trend: increasing
- Data points: 6
- Range: 1000000.00 to 1300000.00


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Model trained on 6 points



Invoking: `train_financial_model` with `[[1,1000000],[2,1100000],[3,1050000],[4,1200000],[5,1250000],[6,1300000]]`


✅ Financial model trained on 6 data points

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Predicted for t=7: 1360000.00
INFO:__main__:Predicted for t=8: 1420000.00



Invoking: `predict_financial_trend` with `7`


📈 Predicted value for t=7: 1360000.00
Invoking: `predict_financial_trend` with `8`


📈 Predicted value for t=8: 1420000.00

> Finished chain.
🎯 Complex Analysis Result:
Agent stopped due to max iterations....

✅ Advanced MCP patterns demonstrated!


## Summary: MCP Tools Integration

Let's summarize the key concepts and best practices for MCP tools integration.

In [10]:
# Summary of MCP Tools Integration
print("📚 MCP TOOLS INTEGRATION SUMMARY")
print("=" * 60)

mcp_summary = {
    "MCP Server Creation": {
        "framework": "FastMCP",
        "pattern": "@mcp.tool() decorator",
        "benefits": ["Simple setup", "Type validation", "Auto documentation"],
        "example": "Financial prediction tools"
    },
    "Tool Conversion": {
        "process": "MCP tools → LangChain tools",
        "wrapper": "Tool() with name, func, description",
        "benefits": ["Agent compatibility", "Error handling", "JSON serialization"],
        "example": "train_financial_model tool"
    },
    "Agent Integration": {
        "method": "create_tool_calling_agent()",
        "features": ["Tool selection", "Context awareness", "Error recovery"],
        "benefits": ["Intelligent tool usage", "Natural language interface"],
        "example": "Financial analysis agent"
    },
    "Best Practices": {
        "data_format": "JSON for complex inputs",
        "error_handling": "Try-catch with meaningful messages",
        "validation": "Type checking and data validation",
        "logging": "Comprehensive logging for debugging"
    }
}

for category, details in mcp_summary.items():
    print(f"\n🔹 {category}:")
    if isinstance(details, dict):
        for key, value in details.items():
            if isinstance(value, list):
                print(f"   {key}: {', '.join(value)}")
            else:
                print(f"   {key}: {value}")

print("\n" + "=" * 60)
print("🎯 KEY TAKEAWAYS")
print("=" * 60)

takeaways = [
    "MCP enables secure, standardized tool integration",
    "FastMCP simplifies server creation with decorators",
    "LangChain tools provide seamless agent integration",
    "Proper error handling is crucial for reliability",
    "JSON serialization handles complex data types",
    "Agents can intelligently chain MCP operations"
]

for i, takeaway in enumerate(takeaways, 1):
    print(f"  {i}. {takeaway}")

print("\n" + "=" * 60)
print("🚀 NEXT STEPS")
print("=" * 60)

next_steps = [
    "Create production MCP servers with real data sources",
    "Implement authentication and security measures",
    "Add more sophisticated financial analysis tools",
    "Build multi-server MCP architectures",
    "Integrate with external APIs and databases",
    "Deploy MCP servers in cloud environments"
]

for i, step in enumerate(next_steps, 1):
    print(f"  {i}. {step}")

print("\n✅ Exercise 9 Complete: MCP Tools Integration Mastered!")
print("🎉 You can now create and consume MCP tools effectively!")

📚 MCP TOOLS INTEGRATION SUMMARY

🔹 MCP Server Creation:
   framework: FastMCP
   pattern: @mcp.tool() decorator
   benefits: Simple setup, Type validation, Auto documentation
   example: Financial prediction tools

🔹 Tool Conversion:
   process: MCP tools → LangChain tools
   wrapper: Tool() with name, func, description
   benefits: Agent compatibility, Error handling, JSON serialization
   example: train_financial_model tool

🔹 Agent Integration:
   method: create_tool_calling_agent()
   features: Tool selection, Context awareness, Error recovery
   benefits: Intelligent tool usage, Natural language interface
   example: Financial analysis agent

🔹 Best Practices:
   data_format: JSON for complex inputs
   error_handling: Try-catch with meaningful messages
   validation: Type checking and data validation
   logging: Comprehensive logging for debugging

🎯 KEY TAKEAWAYS
  1. MCP enables secure, standardized tool integration
  2. FastMCP simplifies server creation with decorators
  3. La